<h1 align=center><font size = 5>Battle of Neighborhoods</font></h1>

### Event venue in Toronto, Canada

## Table of Contents

1. <a href="#item1">Project description: business problem and interested audience in this project.</a>
2. <a href="#item2">Data requirements </a>  
3. <a href="#item3">Methodology</a>  
4. <a href="#item4">Results </a>  
5. <a href="#item5">Discussion and conclusion </a>  




## 1. Project Description:

According to “The 2016 Event Professionals of Tomorrow Report”, selecting an event space is often reported as one of the most important and challenging aspect in the planning process. The location of the event venue is crucial for optimizing event attendance and should be an appropriate place for the target audience.  The type of facility components in the chosen venue should also fit the event. Moreover, suitable lodging accommodations within a reasonable distance of the event and nearby organic event networking opportunities increase the success of the event. 

This project aims to find great venues nearby available event facilities in a city that fulfill the types of events and their requirements. FourSquare data and folium’s visualizing aid as well as data analysis and clustering techniques will solve the problem discussed in this the project.

The interested audiences are event coordinators looking for great venue that not only sets the scene but also comply with requirements of the event and experiences of the participants.  

## 2. Data requiremnts

Data regarding the available event facilities are obtained from https://www.toronto.ca/city-government/data-research-maps/open-data. This database is a compilation of all event spaces within the 44 City wards in Toronto city,  available on a rental basis, for performances, exhibitions, visual arts, screen based performances, heritage events etc. The fields in the dataset are: FACILITY NAME, Full Address, Street #, Street Name, Suite, City, Province, Postal Code, Ward, type of space and ownership of facilities.

For exploring and recommending event facilities across different venues in Toronto city, the FourSquare API interface is also used. The data gathered are then arranged in a dataframe for clustering and visualization.

## 3. Methodology

The first part of the methodology deals with data collection and wrangling to make it suitable for the next stage. Next, location information (Latitude and Longitude) for event facilities would be made to Foursquare API server as HTTP requests using zip codes of the Toronto city streets. Foursquare API search feature would then be used to gather relevant information regarding nearby venues of the city streets. The http request limitations for the number of venues and radius of nearby places are considered.  

Unsupervised K-mean clustering algorithm is later carried out to cluster categories of venues nearby a given event locations. Folium is used to visualize the street venue cluster over an interactive map. These clusters together with the event facilities are used to derive insights for selecting great venues and draw conclusions. 
Thus, the methodology section will have three steps:

##### i. Data collection 
##### ii. Data exploration and wrangling 
##### iii. Modeling



In [172]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from urllib.request import urlopen

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans



print('Libraries imported.')

Libraries imported.


Let's get gather data regarding available event spaces in Toronto city. This is available from Toronto government database at http://opendata.toronto.ca/gcc/MSFC_44%20Wards_Complete_Final.xlsx

In [173]:
link = urlopen("http://opendata.toronto.ca/gcc/MSFC_44%20Wards_Complete_Final.xlsx")
xd = pd.ExcelFile(link)
xd.sheet_names
[u'Database']
df = xd.parse()
#df['Full Address'] = df.apply(lambda x: x['Full Address'][:-8], axis = 1)
df['Full Address'] = [a.replace(b, '').strip() for a, b in zip(df['Full Address'], df['Postal Code'])]
df['Full Address'] = df['Full Address'].str.replace(r',$', '')
df['Postal Code'] = df['Postal Code'].str[:3]
df.fillna(0, inplace=True)
df.head()

,FACILITY NAME,Full Address,Street #,Street Name,Suite,City,Province,Postal Code,Ward,Performance,Exhibition / Visual Arts,Screen Based,Library,Multipurpose,Heritage,OWNERSHIP
0,Thistletown CC,"925 Albion Road, Toronto, ON",925,Albion Road,0,Toronto,ON,M9V,1,1.0,0.0,0.0,0.0,1.0,0.0,City Operated
1,Albion Pool & Health Club,"1485 Albion Road, Toronto, ON",1485,Albion Road,0,Toronto,ON,M9V,1,0.0,0.0,0.0,0.0,1.0,0.0,City Owned
2,Albion Branch (TPL),"1515 Albion Road, Toronto, ON",1515,Albion Road,0,Toronto,ON,M9V,1,1.0,0.0,0.0,1.0,1.0,0.0,City Operated
3,Theatre Francais de Toronto - Centre for Creation,"21 College Street, Office 610, Toronto, ON",21,College Street,Office 610,Toronto,ON,M5G,1,1.0,0.0,0.0,0.0,0.0,1.0,Owned by others
4,Humber Arboretum Gardens,"203 Humber College Boulevard, Toronto, ON",203,Humber College Boulevard,0,Toronto,ON,M9W,1,0.0,0.0,0.0,0.0,1.0,0.0,City Owned


Check the sise of the dataset

In [174]:
df.shape

(1397, 16)

Let's group the event space according to post code (that reflect their neighborhoods) in Toronto city

In [175]:
Facility_grouped=df.groupby('Postal Code')['Performance','Exhibition / Visual Arts','Screen Based','Library','Multipurpose'].count()
Facility_grouped.head()

,Performance,Exhibition / Visual Arts,Screen Based,Library,Multipurpose
Postal Code,,,,,
\nM4,1,1,1,1,1
M1B,9,9,9,9,9
M1C,11,11,11,11,11
M1E,11,11,11,11,11
M1G,5,5,5,5,5


Next we will scrape a wikipedia webpage to get information about the neigborhood and borough to associate with the postal code of the first dataset

In [ ]:
from bs4 import BeautifulSoup
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'xml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df2 = pd.DataFrame(data, columns=['Postal Code', 'Borough', 'Neighbourhood'])
df2 = df2[~df2['Postal Code'].isnull()]  # cleaning rows that contains no information such as Row 0

df2.replace('Not assigned', np.nan, inplace=True) # replace the 'Not assigned' values to nan to facilitate further data cleaning

df2 = df2[df2['Borough'].notnull()] # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

df2['Neighbourhood'].replace(np.nan, df2['Borough'], inplace=True) # A borough with Not assigned neighborhood will have borough as neighborhood

df2=df2.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(','.join).reset_index() # More than one neighborhood for same postal code area are combined

df2.shape

Next we merge the two dataset

In [177]:
df_merged=pd.merge(df, df2, on='Postal Code', how='left')
df_merged['Full Address']=df_merged['Full Address'].str.replace('ON', 'Ontario') 
df_merged.head()

,FACILITY NAME,Full Address,Street #,Street Name,Suite,City,Province,Postal Code,Ward,Performance,Exhibition / Visual Arts,Screen Based,Library,Multipurpose,Heritage,OWNERSHIP,Borough,Neighbourhood
0,Thistletown CC,"925 Albion Road, Toronto, Ontario",925,Albion Road,0,Toronto,ON,M9V,1,1.0,0.0,0.0,0.0,1.0,0.0,City Operated,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
1,Albion Pool & Health Club,"1485 Albion Road, Toronto, Ontario",1485,Albion Road,0,Toronto,ON,M9V,1,0.0,0.0,0.0,0.0,1.0,0.0,City Owned,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
2,Albion Branch (TPL),"1515 Albion Road, Toronto, Ontario",1515,Albion Road,0,Toronto,ON,M9V,1,1.0,0.0,0.0,1.0,1.0,0.0,City Operated,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
3,Theatre Francais de Toronto - Centre for Creation,"21 College Street, Office 610, Toronto, Ontario",21,College Street,Office 610,Toronto,ON,M5G,1,1.0,0.0,0.0,0.0,0.0,1.0,Owned by others,Downtown Toronto,Central Bay Street
4,Humber Arboretum Gardens,"203 Humber College Boulevard, Toronto, Ontario",203,Humber College Boulevard,0,Toronto,ON,M9W,1,0.0,0.0,0.0,0.0,1.0,0.0,City Owned,Etobicoke,Northwest


In [178]:
Facility_by_Borough=df_merged.groupby('Borough').count()
Facility_by_Borough.head()

,FACILITY NAME,Full Address,Street #,Street Name,Suite,City,Province,Postal Code,Ward,Performance,Exhibition / Visual Arts,Screen Based,Library,Multipurpose,Heritage,OWNERSHIP,Neighbourhood
Borough,,,,,,,,,,,,,,,,,
Central Toronto,115,115,115,115,115,115,115,115,115,115,115,115,115,115,115,115,115
Downtown Toronto,443,443,443,443,443,443,443,443,443,443,443,443,443,443,443,443,443
East Toronto,107,107,107,107,107,107,107,107,107,107,107,107,107,107,107,107,107
East York,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57
Etobicoke,86,86,86,86,86,86,86,86,86,86,86,86,86,86,86,86,86


##### Let's explore a bit the resulting dataset

In [180]:
df_target = df_merged.query("(Multipurpose == 1) & (Borough =='Etobicoke')")
df_target.shape

(60, 18)

In [181]:
df_target.head()

,FACILITY NAME,Full Address,Street #,Street Name,Suite,City,Province,Postal Code,Ward,Performance,Exhibition / Visual Arts,Screen Based,Library,Multipurpose,Heritage,OWNERSHIP,Borough,Neighbourhood
0,Thistletown CC,"925 Albion Road, Toronto, Ontario",925,Albion Road,0,Toronto,ON,M9V,1,1.0,0.0,0.0,0.0,1.0,0.0,City Operated,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
1,Albion Pool & Health Club,"1485 Albion Road, Toronto, Ontario",1485,Albion Road,0,Toronto,ON,M9V,1,0.0,0.0,0.0,0.0,1.0,0.0,City Owned,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
2,Albion Branch (TPL),"1515 Albion Road, Toronto, Ontario",1515,Albion Road,0,Toronto,ON,M9V,1,1.0,0.0,0.0,1.0,1.0,0.0,City Operated,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
4,Humber Arboretum Gardens,"203 Humber College Boulevard, Toronto, Ontario",203,Humber College Boulevard,0,Toronto,ON,M9W,1,0.0,0.0,0.0,0.0,1.0,0.0,City Owned,Etobicoke,Northwest
5,Thistletown Baptist Church,"2534 Kipling Avenue, Toronto, Ontario",2534,Kipling Avenue,0,Toronto,ON,M9V,1,0.0,0.0,0.0,0.0,1.0,0.0,Owned by others,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


##### copy the dataset and work with the copied dataframe

In [182]:
df_copied=df_target.copy()

##### Now let's get the locatiion of each event space. 

In [155]:
geolocator = Nominatim()

In [183]:
column_names=['lat','lon']
x=pd.DataFrame(columns=column_names)
for i in np.arange(df_target.shape[0]):
    address_new=df_target.iloc[i,1]
    location = geolocator.geocode(address_new)
    if location:
        la=location.longitude
        lo=location.latitude
        x= x.append({'lat' : 'la' , 'lon' : lo} , ignore_index=True)
    else :
        la=None
        lo=None
        x= x.append({'lat' : 'la' , 'lon' : lo} , ignore_index=True)

In [184]:
column_names=['lat','lon']
y=pd.DataFrame(columns=column_names)
for i in np.arange(df_target.shape[0]):
    address_new=df_target.iloc[i,1]
    location = geolocator.geocode(address_new)
    if location:
        la=location.latitude
        lo=location.longitude
        y= y.append({'lat' : 'la' , 'lon' : lo} , ignore_index=True)
    else :
        la=None
        lo=None
        y= y.append({'lat' : 'la' , 'lon' : lo} , ignore_index=True)

##### Now the resulting coordinates for the venue spaces are

In [185]:
street_cor=pd.DataFrame(columns=column_names)
street_cor['lat']=x['lon']
street_cor['lon']=y['lon']
#street_cor.reset_index(drop=False, inplace=True)
street_cor.head()

,lat,lon
0,43.735450,-79.562527
1,43.739613,-79.580608
2,43.739871,-79.584810
3,43.729845,-79.603792
4,43.744627,-79.583575


#### Add the locations data to the previous dataframe`

In [186]:
latitude=[] #List to collect the latitudes
longitude=[] #List to collect the longitudes

for ind in np.arange(street_cor.shape[0]):
    try:
        latitude=street_cor['lat']
        longitude=street_cor['lon']
     
    except:
        pass   
    
df_copied['Latitude']=latitude #Adding a column in the main dataframe for Latitude
df_copied['Longitude']=longitude #Adding a column in the main dataframe for Longitude
df_copied.head()

,FACILITY NAME,Full Address,Street #,Street Name,Suite,City,Province,Postal Code,Ward,Performance,Exhibition / Visual Arts,Screen Based,Library,Multipurpose,Heritage,OWNERSHIP,Borough,Neighbourhood,Latitude,Longitude
0,Thistletown CC,"925 Albion Road, Toronto, Ontario",925,Albion Road,0,Toronto,ON,M9V,1,1.0,0.0,0.0,0.0,1.0,0.0,City Operated,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.735450,-79.562527
1,Albion Pool & Health Club,"1485 Albion Road, Toronto, Ontario",1485,Albion Road,0,Toronto,ON,M9V,1,0.0,0.0,0.0,0.0,1.0,0.0,City Owned,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739613,-79.580608
2,Albion Branch (TPL),"1515 Albion Road, Toronto, Ontario",1515,Albion Road,0,Toronto,ON,M9V,1,1.0,0.0,0.0,1.0,1.0,0.0,City Operated,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739871,-79.584810
4,Humber Arboretum Gardens,"203 Humber College Boulevard, Toronto, Ontario",203,Humber College Boulevard,0,Toronto,ON,M9W,1,0.0,0.0,0.0,0.0,1.0,0.0,City Owned,Etobicoke,Northwest,43.744627,-79.583575
5,Thistletown Baptist Church,"2534 Kipling Avenue, Toronto, Ontario",2534,Kipling Avenue,0,Toronto,ON,M9V,1,0.0,0.0,0.0,0.0,1.0,0.0,Owned by others,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.747668,-79.586106


#### let's clean the dataset keeping only the relevant infromatiion

In [187]:
df_copied.columns

Index(['FACILITY NAME', 'Full Address', 'Street #', 'Street Name', 'Suite',
       'City', 'Province', 'Postal Code', 'Ward', 'Performance',
       'Exhibition / Visual Arts', 'Screen Based', 'Library', 'Multipurpose',
       'Heritage', 'OWNERSHIP', 'Borough', 'Neighbourhood', 'Latitude',
       'Longitude'],
      dtype='object')

In [188]:
df_copied.drop(['Street Name'], inplace=True, axis=1)
df_copied.drop(['City'], inplace=True, axis=1)
df_copied.drop(['Province'], inplace=True, axis=1)
df_copied.drop(['Street #'], inplace=True, axis=1)
df_copied.drop(['Suite'], inplace=True, axis=1)
df_copied.drop(['Ward'], inplace=True, axis=1)
df_copied.drop(['Heritage'], inplace=True, axis=1)
df_copied.drop(['OWNERSHIP'], inplace=True, axis=1)

In [189]:
df_copied.head()

,FACILITY NAME,Full Address,Postal Code,Performance,Exhibition / Visual Arts,Screen Based,Library,Multipurpose,Borough,Neighbourhood,Latitude,Longitude
0,Thistletown CC,"925 Albion Road, Toronto, Ontario",M9V,1.0,0.0,0.0,0.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.735450,-79.562527
1,Albion Pool & Health Club,"1485 Albion Road, Toronto, Ontario",M9V,0.0,0.0,0.0,0.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739613,-79.580608
2,Albion Branch (TPL),"1515 Albion Road, Toronto, Ontario",M9V,1.0,0.0,0.0,1.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739871,-79.584810
4,Humber Arboretum Gardens,"203 Humber College Boulevard, Toronto, Ontario",M9W,0.0,0.0,0.0,0.0,1.0,Etobicoke,Northwest,43.744627,-79.583575
5,Thistletown Baptist Church,"2534 Kipling Avenue, Toronto, Ontario",M9V,0.0,0.0,0.0,0.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.747668,-79.586106


##### Rearrange the columns in the dataframe so that location related data are at the beggining

In [190]:
columnsTitles = ['FACILITY NAME', 'Full Address', 'Postal Code', 
       'Borough', 'Neighbourhood', 'Latitude', 'Longitude', 'Performance',
       'Exhibition / Visual Arts', 'Screen Based', 'Library', 'Multipurpose']
df_copied.reindex(columns=columnsTitles).head()

,FACILITY NAME,Full Address,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Performance,Exhibition / Visual Arts,Screen Based,Library,Multipurpose
0,Thistletown CC,"925 Albion Road, Toronto, Ontario",M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.735450,-79.562527,1.0,0.0,0.0,0.0,1.0
1,Albion Pool & Health Club,"1485 Albion Road, Toronto, Ontario",M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739613,-79.580608,0.0,0.0,0.0,0.0,1.0
2,Albion Branch (TPL),"1515 Albion Road, Toronto, Ontario",M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739871,-79.584810,1.0,0.0,0.0,1.0,1.0
4,Humber Arboretum Gardens,"203 Humber College Boulevard, Toronto, Ontario",M9W,Etobicoke,Northwest,43.744627,-79.583575,0.0,0.0,0.0,0.0,1.0
5,Thistletown Baptist Church,"2534 Kipling Avenue, Toronto, Ontario",M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.747668,-79.586106,0.0,0.0,0.0,0.0,1.0


##### Let's clean the dataset further so that we keep only those venue space where the location dataset are not null

In [191]:
df_copied = df_copied[df_copied['Latitude'].notnull() & df_copied['Longitude'].notnull()]
df_copied.head()

,FACILITY NAME,Full Address,Postal Code,Performance,Exhibition / Visual Arts,Screen Based,Library,Multipurpose,Borough,Neighbourhood,Latitude,Longitude
0,Thistletown CC,"925 Albion Road, Toronto, Ontario",M9V,1.0,0.0,0.0,0.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.735450,-79.562527
1,Albion Pool & Health Club,"1485 Albion Road, Toronto, Ontario",M9V,0.0,0.0,0.0,0.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739613,-79.580608
2,Albion Branch (TPL),"1515 Albion Road, Toronto, Ontario",M9V,1.0,0.0,0.0,1.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739871,-79.584810
4,Humber Arboretum Gardens,"203 Humber College Boulevard, Toronto, Ontario",M9W,0.0,0.0,0.0,0.0,1.0,Etobicoke,Northwest,43.744627,-79.583575
5,Thistletown Baptist Church,"2534 Kipling Avenue, Toronto, Ontario",M9V,0.0,0.0,0.0,0.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.747668,-79.586106


Use geopy library to get the latitude and longitude values of Toronto City. In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent t_explorer, as shown below.

In [193]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [194]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, street in zip(df_copied['Latitude'], df_copied['Longitude'], df_copied['Full Address']):
    label = '{}'.format(street)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [195]:
CLIENT_ID = 'U1Q44UFGDP5CSB2WVLOPRLVJYDES1KYVHITVR5TNX25TK1NN' # your Foursquare ID
CLIENT_SECRET = 'MTAQ145OYXSDM51QCJNBB3SVEVKI5XCPVQKYYPKBNGO5GNUW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: U1Q44UFGDP5CSB2WVLOPRLVJYDES1KYVHITVR5TNX25TK1NN
CLIENT_SECRET:MTAQ145OYXSDM51QCJNBB3SVEVKI5XCPVQKYYPKBNGO5GNUW


#### Now, let's get the top 100 venues that are in Borough 'Etobicoke' and have a muti purpose facilites within a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.

In [196]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the above function on each location and create a new dataframe called location_venues and display it.

In [197]:
location_venues = getNearbyVenues(names=df_copied['Full Address'],
                                   latitudes=df_copied['Latitude'],
                                   longitudes=df_copied['Longitude']
                                  )

925 Albion Road, Toronto, Ontario
1485 Albion Road, Toronto, Ontario
1515 Albion Road, Toronto, Ontario
203 Humber College Boulevard, Toronto, Ontario
2534 Kipling Avenue, Toronto, Ontario
2580 Kipling Avenue, Toronto, Ontario
175 Mount Olive Drive, Toronto, Ontario
21 Panorama Court, Toronto, Ontario
10 Rampart Road, Toronto, Ontario
34 Riverdale Drive, Toronto, Ontario
2 Rowntree Road, Toronto, Ontario
33 Carlson Court, Toronto, Ontario
650 Dixon Road, Toronto, Ontario
650 Dixon Road, Toronto, Ontario
801 Dixon Road, Toronto, Ontario
850 Humberwood Boulevard, Toronto, Ontario
2170 Kipling Avenue, Toronto, Ontario
555 Rexdale Boulevard, Toronto, Ontario
31 St. Phillips Road, Toronto, Ontario
130 Lloyd Manor Road, Toronto, Ontario
291 Mill Road, Toronto, Ontario
56 Neilson Drive, Toronto, Ontario
475 Rathburn Road, Toronto, Ontario
590 Rathburn Road, Toronto, Ontario
630 Renforth Drive, Toronto, Ontario
399 The West Mall, Toronto, Ontario
450 The West Mall, Toronto, Ontario
10 Toledo R

In [199]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"925 Albion Road, Toronto, Ontario",43.735450,-79.562527,Drupati's Doubles & Roti Shop,43.736222,-79.563471,Caribbean Restaurant
1,"925 Albion Road, Toronto, Ontario",43.735450,-79.562527,Dairy Queen,43.736232,-79.565309,Fast Food Restaurant
2,"925 Albion Road, Toronto, Ontario",43.735450,-79.562527,Shoppers Drug Mart,43.735361,-79.560221,Pharmacy
3,"925 Albion Road, Toronto, Ontario",43.735450,-79.562527,TD Canada Trust,43.736602,-79.562379,Bank
4,"925 Albion Road, Toronto, Ontario",43.735450,-79.562527,Madras Dosa Hut,43.737156,-79.566460,Indian Restaurant
5,"925 Albion Road, Toronto, Ontario",43.735450,-79.562527,Roti-Roti Family Restaurant,43.736241,-79.563531,Caribbean Restaurant
6,"925 Albion Road, Toronto, Ontario",43.735450,-79.562527,Rajdhani Sweets & Restaurant,43.738409,-79.566246,Indian Restaurant
7,"925 Albion Road, Toronto, Ontario",43.735450,-79.562527,Pizza Pizza,43.736619,-79.564600,Pizza Place
8,"925 Albion Road, Toronto, Ontario",43.735450,-79.562527,Brar Pakwan,43.738082,-79.566037,Indian Restaurant
9,"925 Albion Road, Toronto, Ontario",43.735450,-79.562527,Tim Hortons,43.737881,-79.566891,Coffee Shop


##### we now categorize the resulting data according to street. However, before we do that we make a copy of the data set and get rid of the number from the full adress to keep only the street names. we then explore the dataset.

In [200]:
df_cop=location_venues.copy()

In [201]:
def removedigit(string):
    """
    input is a string 
    output is a string with no digits
    """
    return ''.join(ch for ch in string if not ch.isdigit())
df_cop['Street'] = df_cop['Street'].apply(removedigit)
df_cop.head()

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Albion Road, Toronto, Ontario",43.73545,-79.562527,Drupati's Doubles & Roti Shop,43.736222,-79.563471,Caribbean Restaurant
1,"Albion Road, Toronto, Ontario",43.73545,-79.562527,Dairy Queen,43.736232,-79.565309,Fast Food Restaurant
2,"Albion Road, Toronto, Ontario",43.73545,-79.562527,Shoppers Drug Mart,43.735361,-79.560221,Pharmacy
3,"Albion Road, Toronto, Ontario",43.73545,-79.562527,TD Canada Trust,43.736602,-79.562379,Bank
4,"Albion Road, Toronto, Ontario",43.73545,-79.562527,Madras Dosa Hut,43.737156,-79.566460,Indian Restaurant


In [202]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
"10 Rampart Road, Toronto, Ontario",4,4,4,4,4,4
"10 Toledo Road, Toronto, Ontario",1,1,1,1,1,1
"105 Norseman Street, Toronto, Ontario",8,8,8,8,8,8
"130 Lloyd Manor Road, Toronto, Ontario",10,10,10,10,10,10
"1428 Royal York Road, Toronto, Ontario",4,4,4,4,4,4
"1485 Albion Road, Toronto, Ontario",8,8,8,8,8,8
"15 Canmotor Avenue, Toronto, Ontario",14,14,14,14,14,14
"15 Trehorne Drive, Toronto, Ontario",32,32,32,32,32,32
"1515 Albion Road, Toronto, Ontario",9,9,9,9,9,9


In [203]:
# determine the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 118 uniques categories.


In [204]:
location_venues.shape

(523, 7)

##### Analyze Each Street

In [205]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,American Restaurant,Art Gallery,Asian Restaurant,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Beer Store,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Caribbean Restaurant,Casino,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,College Rec Center,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Flower Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gluten-free Restaurant,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,History Museum,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Moving Target,Music Venue,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool Hall,Pub,Racecourse,Racetrack,Recreation Center,Rental Car Location,Rental Service,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Smoothie Shop,Spa,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tanning Salon,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Transportation Service,Video Store,Warehouse Store,Wings Joint,Yoga Studio
0,"925 Albion Road, Toronto, Ontario",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"925 Albion Road, Toronto, Ontario",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"925 Albion Road, Toronto, Ontario",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"925 Albion Road, Toronto, Ontario",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"925 Albion Road, Toronto, Ontario",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


##### Next, let's group rows by Street and by taking the mean of the frequency of occurrence of each category

In [206]:
Etobicoke_grouped = venues_onehot.groupby('Street').mean().reset_index()
Etobicoke_grouped

,Street,American Restaurant,Art Gallery,Asian Restaurant,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Beer Store,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Caribbean Restaurant,Casino,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,College Rec Center,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Flower Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gluten-free Restaurant,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,History Museum,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Moving Target,Music Venue,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool Hall,Pub,Racecourse,Racetrack,Recreation Center,Rental Car Location,Rental Service,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Smoothie Shop,Spa,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tanning Salon,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Transportation Service,Video Store,Warehouse Store,Wings Joint,Yoga Studio
0,"10 Rampart Road, Toronto, Ontario",0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000
1,"10 Toledo Road, Toronto, Ontario",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,1.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000
2,"105 Norseman Street, Toronto, Ontario",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [207]:
Etobicoke_grouped.shape

(46, 119)

In [209]:
# What are the top 5 venues nearby multi-purpose facilities

num_top_venues = 5

for hood in Etobicoke_grouped['Street']:
    print("----"+hood+"----")
    temp = Etobicoke_grouped[Etobicoke_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----10 Rampart Road, Toronto, Ontario----
                venue  freq
0         Supermarket  0.25
1    Asian Restaurant  0.25
2            Pharmacy  0.25
3     Thai Restaurant  0.25
4  Mexican Restaurant  0.00


----10 Toledo Road, Toronto, Ontario----
                        venue  freq
0  Construction & Landscaping   1.0
1         American Restaurant   0.0
2   Middle Eastern Restaurant   0.0
3                         Pub   0.0
4                   Pool Hall   0.0


----105 Norseman Street, Toronto, Ontario----
           venue  freq
0           Park  0.25
1    Yoga Studio  0.12
2  Grocery Store  0.12
3     Playground  0.12
4          Diner  0.12


----130 Lloyd Manor Road, Toronto, Ontario----
               venue  freq
0  Convenience Store   0.1
1            Theater   0.1
2        IT Services   0.1
3               Park   0.1
4     Farmers Market   0.1


----1428 Royal York Road, Toronto, Ontario----
                  venue  freq
0          Intersection  0.25
1             Bookstore  

In [227]:
# Define a function to return the most common venues nearby multi-putpose facilities

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [228]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [229]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = Etobicoke_grouped['Street']

for ind in np.arange(Etobicoke_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(Etobicoke_grouped.iloc[ind, :], num_top_venues)
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"10 Rampart Road, Toronto, Ontario",Thai Restaurant,Asian Restaurant,Pharmacy,Supermarket,Diner,Event Space,Electronics Store,Eastern European Restaurant,Discount Store,Yoga Studio
1,"10 Toledo Road, Toronto, Ontario",Construction & Landscaping,Financial or Legal Service,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Eastern European Restaurant
2,"105 Norseman Street, Toronto, Ontario",Park,Yoga Studio,Skating Rink,Playground,Convenience Store,Diner,Grocery Store,Fried Chicken Joint,French Restaurant,Construction & Landscaping
3,"130 Lloyd Manor Road, Toronto, Ontario",Convenience Store,Hotel,Café,Park,Clothing Store,Theater,IT Services,Coffee Shop,Transportation Service,Farmers Market
4,"1428 Royal York Road, Toronto, Ontario",Gym / Fitness Center,Recreation Center,Intersection,Bookstore,Fast Food Restaurant,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop


##### After investigation of venues nearby event spaces in Toronto London, we could begin clustering them

In [230]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

Etobicoke_grouped_clustering = Etobicoke_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Etobicoke_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([4, 2, 1, 1, 1, 4, 1, 1, 4, 1, 1, 1, 4, 4, 1, 1, 3, 0, 1, 1, 4, 1,
       1, 1, 3, 1, 3, 4, 3, 1, 1, 1, 1, 1, 1, 4, 4, 3, 1, 1, 1, 1, 3, 4,
       1, 1], dtype=int32)

In [231]:
Etobicoke_grouped_clustering.head()

,American Restaurant,Art Gallery,Asian Restaurant,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Beer Store,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Caribbean Restaurant,Casino,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,College Rec Center,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Flower Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gluten-free Restaurant,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,History Museum,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Moving Target,Music Venue,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool Hall,Pub,Racecourse,Racetrack,Recreation Center,Rental Car Location,Rental Service,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Smoothie Shop,Spa,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tanning Salon,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Transportation Service,Video Store,Warehouse Store,Wings Joint,Yoga Studio
0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.25,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
1,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
2,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.00,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125
3,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.100,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.000
4,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.

In [218]:
df_copied.rename(columns={'Full Address':'Street'}, inplace=True)

In [232]:
# add clustering labels
venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_, allow_duplicates=False)

Etobicoke_merged = df_copied

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Etobicoke_merged = Etobicoke_merged.join(venues_sorted.set_index('Street'), on='Street')

Etobicoke_merged.head() 

,FACILITY NAME,Street,Postal Code,Performance,Exhibition / Visual Arts,Screen Based,Library,Multipurpose,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Thistletown CC,"925 Albion Road, Toronto, Ontario",M9V,1.0,0.0,0.0,0.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.735450,-79.562527,1,Indian Restaurant,Caribbean Restaurant,Pharmacy,Fast Food Restaurant,Asian Restaurant,Coffee Shop,Pizza Place,Bank,Thai Restaurant,Supermarket
1,Albion Pool & Health Club,"1485 Albion Road, Toronto, Ontario",M9V,0.0,0.0,0.0,0.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739613,-79.580608,4,Liquor Store,Pizza Place,Video Store,Hardware Store,Pharmacy,Fast Food Restaurant,Fried Chicken Joint,Beer Store,Diner,Electronics Store
2,Albion Branch (TPL),"1515 Albion Road, Toronto, Ontario",M9V,1.0,0.0,0.0,1.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739871,-79.584810,4,Fast Food Restaurant,Fried Chicken Joint,Grocery Store,Pharmacy,Coffee Shop,Sandwich Place,Beer Store,Pizza Place,Hardware Store,Furniture / Home Store
4,Humber Arboretum Gardens,"203 Humber College Boulevard, Toronto, Ontario",M9W,0.0,0.0,0.0,0.0,1.0,Etobicoke,Northwest,43.744627,-79.583575,4,Fast Food Restaurant,Fried Chicken Joint,Pharmacy,Park,Liquor Store,Japanese Restaurant,Beer Store,Pizza Place,Grocery Store,Hardware Store
5,Thistletown Baptist Church,"2534 Kipling Avenue, Toronto, Ontario",M9V,0.0,0.0,0.0,0.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.747668,-79.586106,3,Coffee Shop,Park,Yoga Studio,Concert Hall,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner


#### Finally, let's visualize the resulting clusters

In [234]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Etobicoke_merged['Latitude'], Etobicoke_merged['Longitude'], Etobicoke_merged['Street'], Etobicoke_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Cluster

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 1

In [236]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 0, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,Street,Screen Based,Library,Multipurpose,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"2580 Kipling Avenue, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.749735,-79.596359,0,Deli / Bodega,Shopping Mall,Yoga Studio,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Dance Studio,Department Store,Dessert Shop,Diner


#### Cluster 2

In [237]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 1, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,Street,Screen Based,Library,Multipurpose,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"925 Albion Road, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.735450,-79.562527,1,Indian Restaurant,Caribbean Restaurant,Pharmacy,Fast Food Restaurant,Asian Restaurant,Coffee Shop,Pizza Place,Bank,Thai Restaurant,Supermarket
7,"175 Mount Olive Drive, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.746311,-79.581547,1,Park,Caribbean Restaurant,Business Service,Yoga Studio,Construction & Landscaping,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
10,"34 Riverdale Drive, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.752519,-79.584733,1,Convenience Store,Construction & Landscaping,Coffee Shop,Business Service,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store
11,"2 Rowntree Road, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.690239,-79.583648,1,Hotel,American Restaurant,Coffee Shop,Steakhouse,BBQ Joint,Bar,Breakfast Spot,Japanese Restaurant,Mediterranean Restaurant,Pizza Place
12,"33 Carlson Court, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,Northwest,43.688923,-79.577871,1,Rental Car Location,Sushi Restaurant,Hotel,Restaurant,Coffee Shop,American Restaurant,Japanese Restaurant,Thai Restaurant,Bar,Sandwich Place
13,"650 Dixon Road, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,Northwest,43.688923,-79.577871,1,Hotel,American Restaurant,Rental Car Location,Steakhouse,Coffee Shop,Sushi Restaurant,Restaurant,Fast Food Restaurant,Wings Joint,Deli / Bodega
14,"650 Dixon Road, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,Northwest,43.686541,-79.587166,1,Hotel,American Restaurant,Rental Car Location,Steakhouse,Coffee Shop,Sushi Restaurant,Restaurant,Fast Food Restaurant,Wings Joint,Deli / Bodega
17,"2170 Kipling Avenue, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,Northwest,43.715209,-79.604134,1,American Restaurant,Racecourse,Coffee Shop,Restaurant,Casino,Café,Construction & Landscaping,Dance Studio,Deli / Bodega,Department Store
23,"31 St. Phillips Road, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,Westmount,43.650081,-79.582889,1,Baseball Field,College Rec Center,Bank,Garden,Fish & Chips Shop,Electronics Store,Stadium,Grocery Store,Bar,Asian Restaurant
26,"130 Lloyd Manor Road, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar...",43.647058,-79.568018,1,Convenience Store,Hotel,Café,Park,Clothing Store,Theater,IT Services,Coffee Shop,Transportation Service,Farmers Market


#### Cluster 3

In [238]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 2, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,Street,Screen Based,Library,Multipurpose,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,"10 Toledo Road, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...",43.637905,-79.496403,2,Construction & Landscaping,Financial or Legal Service,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Eastern European Restaurant


#### Cluster 4

In [239]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 3, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,Street,Screen Based,Library,Multipurpose,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,"2534 Kipling Avenue, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.747668,-79.586106,3,Coffee Shop,Park,Yoga Studio,Concert Hall,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
15,"801 Dixon Road, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,Northwest,43.729175,-79.619275,3,Park,Bus Station,Yoga Studio,Concert Hall,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
30,"590 Rathburn Road, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...",43.683395,-79.541127,3,Park,Moving Target,Baseball Field,Yoga Studio,Fast Food Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
47,"3326 Bloor Street West, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.634084,-79.492077,3,Park,Baseball Field,Yoga Studio,Fast Food Restaurant,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
48,"4050 Bloor Street West, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar...",43.630916,-79.503786,3,Park,Moving Target,Yoga Studio,Gym,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
49,"36 Brentwood Road North, Toronto, Ontario",0.0,1.0,1.0,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.642709,-79.508802,3,Park,Art Gallery,Yoga Studio,Fast Food Restaurant,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner


#### Cluster 5

In [240]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 4, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,Street,Screen Based,Library,Multipurpose,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"1485 Albion Road, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739613,-79.580608,4,Liquor Store,Pizza Place,Video Store,Hardware Store,Pharmacy,Fast Food Restaurant,Fried Chicken Joint,Beer Store,Diner,Electronics Store
2,"1515 Albion Road, Toronto, Ontario",0.0,1.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739871,-79.584810,4,Fast Food Restaurant,Fried Chicken Joint,Grocery Store,Pharmacy,Coffee Shop,Sandwich Place,Beer Store,Pizza Place,Hardware Store,Furniture / Home Store
4,"203 Humber College Boulevard, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,Northwest,43.744627,-79.583575,4,Fast Food Restaurant,Fried Chicken Joint,Pharmacy,Park,Liquor Store,Japanese Restaurant,Beer Store,Pizza Place,Grocery Store,Hardware Store
8,"21 Panorama Court, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.734794,-79.588682,4,Pizza Place,Grocery Store,Bus Line,Concert Hall,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
9,"10 Rampart Road, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.731250,-79.559720,4,Thai Restaurant,Asian Restaurant,Pharmacy,Supermarket,Diner,Event Space,Electronics Store,Eastern European Restaurant,Discount Store,Yoga Studio
16,"850 Humberwood Boulevard, Toronto, Ontario",0.0,1.0,1.0,Etobicoke,Northwest,43.720756,-79.572444,4,Fast Food Restaurant,Pizza Place,Department Store,Sandwich Place,Concert Hall,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner
20,"555 Rexdale Boulevard, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,Northwest,43.638157,-79.579916,4,Pizza Place,Pharmacy,Park,Beer Store,Café,Shopping Plaza,Yoga Studio,Event Space,Dance Studio,Deli / Bodega
27,"291 Mill Road, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...",43.639932,-79.572752,4,Liquor Store,Beer Store,Pizza Place,Pharmacy,Park,Café,Shopping Plaza,Convenience Store,Food Truck,Food Court
28,"56 Neilson Drive, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...",43.673954,-79.560480,4,Liquor Store,Coffee Shop,Pharmacy,Supermarket,Fast Food Restaurant,Convenience Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop
33,"399 The West Mall, Toronto, Ontario",0.0,0.0,1.0,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...",43.665125,-79.528947,4,Pharmacy,Grocery Store,Skating Rink,Park,Bank,Shopping Mall,Café,Yoga Studio,Discount Store,Eastern European Restaurant


## 4. Results

An effort has been made to facilitate decision regarding even space in Toronto city. The procedure was to get information regarding all available facilities such as type of facility and full address. This was combined with another dataset that contained neighborhood and borough information. The latter dataset helps in the decision process to enable selection of facilities in each region. This decision information together with the type of event space is used to find popular venues near the selected regions that consist of the desired venue spaces.

Finally, it was possible to analyze the results according to the five clusters. Clusters gave out popular venues that could help achieve great event space. One could see, especially for cluster number 2, 4, and 5 that a pattern emerges in direct relation to the busy-ness of the venue and the type of services available. Particularly cluster 4 is ideal places for artist who would like to perform in Etobicoke region. This is owning to the presence of great opportunities for organic event networking.

## 5. Discussion and conclusion

In conclusion, this project successfully find and cluster great venues nearby available event facilities in a city that fulfill the types of events and their requirements.In addition, emerging patterns in cluster could be used as a prime indicators to select great venue spaces that facilitated decision for event coordinators. 